# EDA Covid

In [2]:
# !curl -o /tmp/postgresql-42.1.4.jar https://jdbc.postgresql.org/download/postgresql-42.1.4.jar

In [3]:
# !pyspark --driver-class-path /tmp/postgresql-42.1.4.jar --jars /tmp/postgresql-42.1.4.jar

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import DataFrameReader, SQLContext, SparkSession

import os

## Mode 1

In [3]:
spark = SparkSession \
    .builder \
    .appName("DataSUS") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.2.22.jar") \
    .master('spark://wilian-ubuntu:7077') \
    .getOrCreate()

In [4]:
spark.sql("SHOW TABLES")

DataFrame[database: string, tableName: string, isTemporary: boolean]

In [8]:
df = spark.read.format("jdbc"). \
options(
         url='jdbc:postgresql://localhost:5432/datasus', # jdbc:postgresql://<host>:<port>/<database>
         dbtable='sia_pa_rs',
         user='postgres',
         password='postgres',
         driver='org.postgresql.Driver',
        #partitionColumn='index',
        ).\
load()

In [9]:
df.printSchema()

root
 |-- index: long (nullable = true)
 |-- pa_gestao: long (nullable = true)
 |-- pa_condic: string (nullable = true)
 |-- pa_ufmun: long (nullable = true)
 |-- pa_regct: long (nullable = true)
 |-- pa_incout: long (nullable = true)
 |-- pa_incurg: long (nullable = true)
 |-- pa_tpups: long (nullable = true)
 |-- pa_tippre: long (nullable = true)
 |-- pa_mn_ind: string (nullable = true)
 |-- pa_cnpjcpf: long (nullable = true)
 |-- pa_cnpjmnt: long (nullable = true)
 |-- pa_cnpj_cc: long (nullable = true)
 |-- pa_mvm: long (nullable = true)
 |-- pa_cmp: long (nullable = true)
 |-- pa_proc_id: long (nullable = true)
 |-- pa_tpfin: long (nullable = true)
 |-- pa_subfin: long (nullable = true)
 |-- pa_nivcpl: long (nullable = true)
 |-- pa_docorig: string (nullable = true)
 |-- pa_autoriz: string (nullable = true)
 |-- pa_cnsmed: long (nullable = true)
 |-- pa_cbocod: string (nullable = true)
 |-- pa_motsai: long (nullable = true)
 |-- pa_obito: long (nullable = true)
 |-- pa_encerr: lon

In [12]:
# df.sample(.5).show(5)

In [7]:
# spark.sql('select count(1) from datasus.sia_pa_rs').show()

In [ ]:
# df.count()

In [ ]:
df.select("*").toPandas()

In [ ]:
df.select('*').collect()

In [ ]:
# df.show(1)
# df.take(1)


In [6]:
df.createOrReplaceTempView("tableA")

In [7]:
teenagerDF = spark.sql("SELECT min(PA_IDADE) FROM tableA WHERE PA_CIDPRI = 'H57'")
teenagerDF.show()

+-------------+
|min(PA_IDADE)|
+-------------+
|            0|
+-------------+



In [8]:
teenagerDF = spark.sql("SELECT max(PA_IDADE) FROM tableA WHERE PA_CIDPRI = 'H57'")
teenagerDF.show()

+-------------+
|max(PA_IDADE)|
+-------------+
|           76|
+-------------+



In [ ]:
# spark.sql("SELECT count(*) from tableA").show()

## Mode 2

In [2]:
# sparkClassPath = os.getenv('SPARK_CLASSPATH', '/opt/spark/jars/postgresql-42.2.22.jar')

# # Populate configuration
# conf = SparkConf()
# conf.setAppName('DataSUSCtx')
# conf.set('spark.jars', 'file:%s' % sparkClassPath)
# conf.set('spark.executor.extraClassPath', sparkClassPath)
# conf.set('spark.driver.extraClassPath', sparkClassPath)

# conf.set('spark.driver.cores', '1')
# conf.set('spark.executor.cores', '1')
# conf.set('spark.driver.memory', '4G')
# conf.set('spark.executor.memory', '4G')

# conf.set('spark.master', 'spark://wilian-ubuntu:7077')

In [3]:
# spark = SparkSession.builder.getOrCreate()

In [3]:
# sc = SparkContext(conf=conf)

# sqlContext = SQLContext(sc)

In [5]:
spark = SparkSession \
    .builder \
    .appName("DataSUS") \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.2.22.jar") \
    .master('spark://wilian-ubuntu:7077') \
    .getOrCreate()

In [6]:
sc = spark.sparkContext

sqlContext=SQLContext(sc)

In [7]:
spark.sql("SHOW TABLES")

DataFrame[database: string, tableName: string, isTemporary: boolean]

In [8]:
url = 'postgresql://127.0.0.1:5432/datasus'
properties = {'user':'postgres', 'password':'postgres', 'driver':'org.postgresql.Driver'}

In [9]:
df = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='sia_pa_rs', properties=properties)

In [7]:
df.printSchema()

root
 |-- index: long (nullable = true)
 |-- pa_gestao: long (nullable = true)
 |-- pa_condic: string (nullable = true)
 |-- pa_ufmun: long (nullable = true)
 |-- pa_regct: long (nullable = true)
 |-- pa_incout: long (nullable = true)
 |-- pa_incurg: long (nullable = true)
 |-- pa_tpups: long (nullable = true)
 |-- pa_tippre: long (nullable = true)
 |-- pa_mn_ind: string (nullable = true)
 |-- pa_cnpjcpf: long (nullable = true)
 |-- pa_cnpjmnt: long (nullable = true)
 |-- pa_cnpj_cc: long (nullable = true)
 |-- pa_mvm: long (nullable = true)
 |-- pa_cmp: long (nullable = true)
 |-- pa_proc_id: long (nullable = true)
 |-- pa_tpfin: long (nullable = true)
 |-- pa_subfin: long (nullable = true)
 |-- pa_nivcpl: long (nullable = true)
 |-- pa_docorig: string (nullable = true)
 |-- pa_autoriz: string (nullable = true)
 |-- pa_cnsmed: long (nullable = true)
 |-- pa_cbocod: string (nullable = true)
 |-- pa_motsai: long (nullable = true)
 |-- pa_obito: long (nullable = true)
 |-- pa_encerr: lon

In [ ]:
df.columns

In [9]:
print(spark)

In [10]:
print(sc)
print(sqlContext)

<SparkContext master=local[*] appName=DataSUSCtx>


In [11]:
# Resilient Distributed Dataframe (RDD)

In [12]:
df.rdd

MapPartitionsRDD[4] at javaToPython at NativeMethodAccessorImpl.java:0

In [13]:
df.rdd.getNumPartitions()

1

In [14]:
df = df.repartition(4)

In [15]:
df.rdd.getNumPartitions()

4

In [12]:
# df.collect()

# Py4JJavaError: An error occurred while calling o56.collectToPython.
# org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (wilian-ubuntu executor driver): java.lang.OutOfMemoryError: Java heap space
# Driver stacktrace:

In [13]:
df.where('index == 2250802').show()

+-------+---------+---------+--------+--------+---------+---------+--------+---------+---------+--------------+----------+----------+------+------+----------+--------+---------+---------+----------+-------------+---------------+---------+---------+--------+---------+---------+-------+---------+---------+---------+---------+---------+--------+--------+--------+----------+-------+----------+---------+---------+---------+---------+---------+--------+--------+----------+----------+---------+---------+---------+-------+-------+--------+--------+--------+---------+--------+------+----------+
|  index|pa_gestao|pa_condic|pa_ufmun|pa_regct|pa_incout|pa_incurg|pa_tpups|pa_tippre|pa_mn_ind|    pa_cnpjcpf|pa_cnpjmnt|pa_cnpj_cc|pa_mvm|pa_cmp|pa_proc_id|pa_tpfin|pa_subfin|pa_nivcpl|pa_docorig|   pa_autoriz|      pa_cnsmed|pa_cbocod|pa_motsai|pa_obito|pa_encerr|pa_perman|pa_alta|pa_transf|pa_cidpri|pa_cidsec|pa_cidcas|pa_catend|pa_idade|idademin|idademax|pa_flidade|pa_sexo|pa_racacor|pa_munpcn|pa_qt

In [14]:
df.select('PA_IDADE').distinct().count()

132

In [20]:
# df.limit(10).toPandas()

In [16]:
# df.select(
#     'pa_gestao',
#     'pa_ufmun',
#     'pa_cbocod',
#     'pa_cidpri',
#     'pa_cidsec',
# ).limit(10).toPandas()

DataFrame[pa_gestao: bigint, pa_ufmun: bigint, pa_cbocod: string, pa_cidpri: string, pa_cidsec: string]

In [14]:
# df.take(1)

In [9]:
df.createOrReplaceTempView("tableA")

In [10]:
teenagerDF = spark.sql("SELECT min(PA_IDADE) FROM tableA WHERE PA_CIDPRI = 'H57'")
teenagerDF.show()

+-------------+
|min(PA_IDADE)|
+-------------+
|            0|
+-------------+



In [14]:
teenagerDF.summary()

DataFrame[summary: string, min(PA_IDADE): string]

In [16]:
teenagerDF.toPandas()

,min(PA_IDADE)
0,0


In [9]:
# df.show(1, vertical=True)

In [12]:
%%time
df.select("PA_CIDPRI", "PA_CIDSEC").describe().show()

+-------+-------------------+---------+
|summary|          PA_CIDPRI|PA_CIDSEC|
+-------+-------------------+---------+
|  count|           40722508| 40722508|
|   mean|0.04845269043950959|      0.0|
| stddev|  7.336797206585033|      0.0|
|    min|               0000|        0|
|    max|               Z999|     Z940|
+-------+-------------------+---------+



In [17]:
df.where('PA_CIDPRI != "0000" and PA_CIDSEC != "0000" and PA_CIDSEC != "0"').select("PA_CIDPRI", "PA_CIDSEC").show()

+---------+---------+
|PA_CIDPRI|PA_CIDSEC|
+---------+---------+
|     N180|     N189|
|     N180|     N189|
|     F845|        0|
|     F711|        0|
|     F848|        0|
|     F845|        0|
|     F840|        0|
|     C189|        0|
|      C20|        0|
|      F99|        0|
|     F985|        0|
|     F840|        0|
|     Z000|        0|
|     H353|        0|
|      B34|        0|
|     R529|        0|
|     H250|        0|
|     D649|        0|
|      S80|        0|
|     N649|        0|
+---------+---------+
only showing top 20 rows



In [8]:
# dataCollect = df.groupby('PA_CIDPRI').count().collect() #.sort('count', ascending=False).collect() #.show()

In [9]:
# print(dataCollect)

In [10]:
# for row in dataCollect:
#    print(row)
    #print(row['dept_name'] + "," +str(row['dept_id']))

In [ ]:
df.groupby('PA_CIDSEC').count().show() #.sort('count', ascending=False).show()

In [10]:
df.count()

40722508

In [13]:
from pyspark.sql.functions import expr

# df.selectExpr('add_one(v1)').show()
df.select(expr('PA_CIDPRI') == "H57").show()

+-----------------+
|(PA_CIDPRI = H57)|
+-----------------+
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
|            false|
+-----------------+
only showing top 20 rows



In [8]:
df.filter(df["PA_CIDPRI"] == "H57").show()

+-------+---------+---------+--------+--------+---------+---------+--------+---------+---------+--------------+--------------+----------+------+------+----------+--------+---------+---------+----------+-------------+---------------+---------+---------+--------+---------+---------+-------+---------+---------+---------+---------+---------+--------+--------+--------+----------+-------+----------+---------+---------+---------+---------+---------+--------+--------+----------+----------+---------+---------+---------+-------+-------+--------+--------+--------+---------+--------+------+----------+
|  index|pa_gestao|pa_condic|pa_ufmun|pa_regct|pa_incout|pa_incurg|pa_tpups|pa_tippre|pa_mn_ind|    pa_cnpjcpf|    pa_cnpjmnt|pa_cnpj_cc|pa_mvm|pa_cmp|pa_proc_id|pa_tpfin|pa_subfin|pa_nivcpl|pa_docorig|   pa_autoriz|      pa_cnsmed|pa_cbocod|pa_motsai|pa_obito|pa_encerr|pa_perman|pa_alta|pa_transf|pa_cidpri|pa_cidsec|pa_cidcas|pa_catend|pa_idade|idademin|idademax|pa_flidade|pa_sexo|pa_racacor|pa_munp

In [11]:
h57 = df.where('PA_CIDPRI like "H57%"') #.show()

In [12]:
h57.write.saveAsTable("sample_h57")

In [18]:
sqlContext.sql("show tables").show()

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
| default|sample_h57|      false|
|        |    tablea|       true|
+--------+----------+-----------+



In [15]:
# Properties to connect to the database,
# the JDBC driver is part of our build.sbt
dbConnectionUrl = "jdbc:postgresql://localhost:5432/datasus"
mode = "overwrite"
table = "h57"
properties = {"user": "postgres","password": "postgres","driver": "org.postgresql.Driver"}
# Write in a table called ch02
h57.write.jdbc(url=dbConnectionUrl, table=table, mode=mode, properties=properties)

In [16]:
sqlContext.sql("select count(1) from sample_h57").show()

+--------+
|count(1)|
+--------+
|   61793|
+--------+



In [17]:
spark.sql("SHOW TABLES")

DataFrame[database: string, tableName: string, isTemporary: boolean]

In [18]:
# df.select("*").toPandas()

# End

In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [ ]:
# format2018df = load_data_using_2018_format(spark,"2018*.csv")

# format2015df = load_data_using_2015_format(spark,"2015*.csv")

# format2006df = load_data_using_2006_format(spark,"200*.csv")

# format2006df2 = load_data_using_2006_format(spark,"2012*.csv")

# df = format2018df.unionByName(format2015df).unionByName(format2006df).unionByName(format2006df2)
